In [1]:
###f01###
# -*- coding: utf-8 -*-
import warnings
import os.path
from code.refactor.common import (get_setting_from_cfg, get_setting_path, filter_df_by_cate_id,get_future_sales_feature,\
                                  dummy_features, model_predict, get_weekly_df,get_future_condition_sales_feature,gen_train_valid_by_date,\
                                  generate_promotion_future, get_hour, save_object, add_pv,get_lowest_n,generate_cv_train_valid_set,\
                                  generate_default_values_by_dic,fill_col_with_default, week2month,trans_band_to_int,get_future_condition_sales_feature_stage2,\
                                  loadSettingsFromYamlFile,createSeasonalityFeatures,createLevel3Features,createSeasonalityDecomposeFeatures,\
                                 calculateNationalRolling,calculateRolling,calculateLagging,createDateFeatures,splitTimeWindow,calculateSimilarRolling,calculateStockFeatures,\
                                 process_rdc,clean_data,add_cols,get_dd_price,agg_dd_price,get_bundle_feat,agg_bd_price,process_feature,\
                                 agg_feature,prep_data,calc_weighted_price,agg_feature_day,get_column_by_type,object2Float,object2Int)
from code.refactor.fdc_flow import  filter_non_price_fill_it
import pandas as pd
import xgboost as xgb
import numpy as np
import datetime
warnings.filterwarnings("ignore", category=DeprecationWarning)

#p1_path = 'tmp/0411_2018_brand/p1_0411.csv' 

path = 'tmp/data/shishang'
suffix = '.da'
item = 'p1'
for_what = ['train', 'predict']
cate_list = [2589]
#cate_list = [2584,12029]
scenarioSettingsPath = 'code/refactor/ow_scenario.yaml'
scenario = loadSettingsFromYamlFile(scenarioSettingsPath)
area_rdc_map = pd.read_csv('/home/ubuntu/yulong/promotion_offline/tmp/ow_deploy_single/area_rdc_mapping.csv')
import ow_f01


/home/ubuntu/lijingjie/ljj/local/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [5]:
for fw in for_what:
    for cate in cate_list:
        print "output and save: %s_p1_%s"%(str(cate),fw)
        p1_path = os.path.join(path, str(cate)+'/'+str(cate)+'_'+item+suffix)

        p1_used_header=['productkey', 'promotionkey', 'startdatetime', 'enddatetime', 'jdprice', 'syntheticgrossprice', 'promotiondesc', 'promotiondesc_flag', 'promotiontype', 'promotionsubtype',
	                'areatypearray', 'tokenflag', 'directdiscount_discount', 'directdiscount_availabilitynumber', 'bundle_subtype1_threshold', 'bundle_subtype1_giveaway',
	                'bundle_subtype4_threshold1', 'bundle_subtype4_giveaway1', 'bundle_subtype4_threshold2', 'bundle_subtype4_giveaway2', 'bundle_subtype4_threshold3',
	                'bundle_subtype4_giveaway3', 'bundle_subtype2_threshold', 'bundle_subtype2_giveaway', 'bundle_subtype2_maximumgiveaway', 'bundle_subtype15_thresholdnumber1',
	                'bundle_subtype15_giveawayrate1', 'bundle_subtype15_thresholdnumber2', 'bundle_subtype15_giveawayrate2', 'bundle_subtype15_thresholdnumber3',
	                'bundle_subtype15_giveawayrate3', 'bundle_subtype6_thresholdnumber', 'bundle_subtype6_freenumber', 'suit_maxvaluepool', 'suit_minvaluepool', 'suit_avgvaluepool',
	                'suit_discount', 'directdiscount_saleprice', 'bundle_subtype1_percent', 'bundle_subtype4_percent', 'bundle_subtype2_percent', 'bundle_subtype15_percent',
	                'bundle_subtype6_percent', 'suit_percent', 'allpercentdiscount', 'mainproductkey', 'hierarchylevel3key', 'createdate', 'statuscode', 'dt']
        period_promo_raw = pd.read_csv(p1_path,sep='\t',header=None)
        period_promo_raw.columns=p1_used_header

        period_promo_raw_clean = period_promo_raw
        train_pred_gate = fw   # 'train'
        if cate in [2589]:
            pred_date = pd.to_datetime('2018-04-01')
            scenario['lookforwardPeriodDays'] = 7
        if cate in [2584,12029]:
            pred_date = pd.to_datetime('2018-05-07')
            scenario['lookforwardPeriodDays'] = 7
        else:
            pred_date = pd.to_datetime('2018-02-01')
            scenario['lookforwardPeriodDays'] = 10
        print pred_date
        f01_output = ow_f01.generate_f01_promo(area_rdc_map, period_promo_raw_clean,scenario, train_pred_gate, ForecastStartDate=pred_date)
        f01_output.to_csv(os.path.join(path, str(cate)+'/'+train_pred_gate+'_'+item+'_'+str(cate)+'.csv'),index=False)


output and save: 2584_p1_train
convert to Float columns:
allpercentdiscount
convert to Float columns:
bundle_subtype15_giveawayrate1
convert to Float columns:
bundle_subtype15_giveawayrate2
convert to Float columns:
bundle_subtype15_giveawayrate3
convert to Float columns:
bundle_subtype15_percent
convert to Float columns:
bundle_subtype15_thresholdnumber1
convert to Float columns:
bundle_subtype15_thresholdnumber2
convert to Float columns:
bundle_subtype15_thresholdnumber3
convert to Float columns:
bundle_subtype1_giveaway
convert to Float columns:
bundle_subtype1_percent
convert to Float columns:
bundle_subtype1_threshold
convert to Float columns:
bundle_subtype2_giveaway
convert to Float columns:
bundle_subtype2_maximumgiveaway
convert to Float columns:
bundle_subtype2_percent
convert to Float columns:
bundle_subtype2_threshold
convert to Float columns:
bundle_subtype4_giveaway1
convert to Float columns:
bundle_subtype4_giveaway2
convert to Float columns:
bundle_subtype4_giveaway3
co

/home/ubuntu/lijingjie/ljj/local/lib/python2.7/site-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
code/refactor/common.py:2111: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  period_promo_raw['rdc_list'] = period_promo_raw['areatypearray'].apply(lambda x: extract_rdc(x, area_rdc_dict))
/home/ubuntu/lijingjie/ljj/local/lib/python2.7/site-packages/pandas/core/generic.py:4355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.

('| - Before filter: ', 1002561)
('| - Filtered tokens: ', 1000791)
('| - Filtered descriptions: ', 999071)
('| - Filtered flash sale available units', 998985)
('| - Filtered non-national promo: ', 998939)
| | - Add additional columns
| | - Calculate dd price per promotion
| | - Calculate dd price per period
| | - Generate bundle features
| | - Calculate bundle price per period
| - Reformat date
| | - process features
| | - aggregate features per period
| | - prep data for Step 3
| | - fix indicators on days with end time 00:00 or start time 23:59
| | - calculate weighted price per day
| | - aggregate features per day
f01 shape
(6326406, 23)
output and save: 12029_p1_train
convert to Float columns:
allpercentdiscount
convert to Float columns:
bundle_subtype15_giveawayrate1
convert to Float columns:
bundle_subtype15_giveawayrate2
convert to Float columns:
bundle_subtype15_giveawayrate3
convert to Float columns:
bundle_subtype15_percent
convert to Float columns:
bundle_subtype15_threshol

('| - Before filter: ', 32295)
('| - Filtered tokens: ', 32272)
('| - Filtered descriptions: ', 32151)
('| - Filtered flash sale available units', 32147)
('| - Filtered non-national promo: ', 32147)
| | - Add additional columns
| | - Calculate dd price per promotion
| | - Calculate dd price per period
| | - Generate bundle features
| | - Calculate bundle price per period
| - Reformat date
| | - process features
| | - aggregate features per period
| | - prep data for Step 3
| | - fix indicators on days with end time 00:00 or start time 23:59
| | - calculate weighted price per day
| | - aggregate features per day
f01 shape
(2650256, 23)


In [6]:
###train###
import warnings
import os.path
from code.refactor.common import (get_setting_from_cfg, get_setting_path, filter_df_by_cate_id,get_future_sales_feature,\
                                  dummy_features, model_predict, get_weekly_df,get_future_condition_sales_feature,gen_train_valid_by_date,\
                                  generate_promotion_future, get_hour, save_object, add_pv,get_lowest_n,generate_cv_train_valid_set,\
                                  generate_default_values_by_dic,fill_col_with_default, week2month,trans_band_to_int,get_future_condition_sales_feature_stage2,\
                                  loadSettingsFromYamlFile,createSeasonalityFeatures,createLevel3Features,createSeasonalityDecomposeFeatures,\
                                 calculateNationalRolling,calculateRolling,calculateLagging,createDateFeatures,splitTimeWindow,calculateSimilarRolling,calculateStockFeatures,\
                                 process_rdc,clean_data,add_cols,get_dd_price,agg_dd_price,get_bundle_feat,agg_bd_price,process_feature,\
                                 agg_feature,prep_data,calc_weighted_price,agg_feature_day,get_column_by_type,object2Float,object2Int)
from code.refactor.fdc_flow import  filter_non_price_fill_it
import pandas as pd
import xgboost as xgb
import numpy as np
import datetime
warnings.filterwarnings("ignore", category=DeprecationWarning)


In [7]:
path = 'tmp/data/shishang'
result_path = 'tmp/data/shishang'
cate_list = [2584]
#cate_list = [7052, 7054, 870]
scenarioSettingsPath = 'code/refactor/ow_scenario.yaml'
scenario = loadSettingsFromYamlFile(scenarioSettingsPath)
'''online analysis'''
#scenario['use_cart_feature'] = False
#scenario['use_promo_features'] = False
holidays_df=pd.read_csv('/home/ubuntu/yulong/promotion_offline/tmp/ow_deploy_single/holidays.csv')
area_rdc_map = pd.read_csv('/home/ubuntu/yulong/promotion_offline/tmp/ow_deploy_single/area_rdc_mapping.csv')
import ow_train


In [ ]:
for cate in cate_list:
    if cate in [2589]:
        pred_date = pd.to_datetime('2018-04-01')
        scenario['lookforwardPeriodDays'] = 7
    if cate in [2584,12029]:
        pred_date = pd.to_datetime('2018-05-07')
        scenario['lookforwardPeriodDays'] = 7
    else:
        pred_date = pd.to_datetime('2018-02-01')
        scenario['lookforwardPeriodDays'] = 10
        
    pro_canlender_path = os.path.join(path, str(cate)+'/'+str(cate)+'_'+'p2.da')
    promoCalendarDf = pd.read_csv(pro_canlender_path,sep='\t',header=None)
    promoCalendarDf.columns=['ProductKey', 'Date', 'HierarchyLevel3Key', 'PromotionCount', 'bundlecount', 'MaxDiscount', 'MinDiscount', 'AvgDiscount', 'MaxSyntheticDiscountA',
                 'MinSyntheticDiscountA', 'AvgSyntheticDiscountA', 'MaxBundleDiscount', 'MinBundleDiscount', 'AvgBundleDiscount', 'MaxDirectDiscount', 'MinDirectDiscount',
                 'AvgDirectDiscount', 'MaxFreegiftDiscount', 'MinFreegiftDiscount', 'AvgFreegiftDiscount', 'SyntheticGrossPrice', 'promotionkey', 'promotiontype',
                 'promotionsubtype', 'syntheticgrossprice_vb', 'jdprice', 'syntheticdiscounta_vb', 'durationinhours', 'daynumberinpromotion', 'bundleflag', 'directdiscountflag',
                 'freegiftflag', 'suitflag', 'numberproducts', 'numberhierarchylevel1', 'numberhierarchylevel2', 'numberhierarchylevel3', 'strongmark', 'stockprice', 'dt']

    p1_out_path = os.path.join(path, str(cate)+'/'+'train'+'_p1_'+str(cate)+'.csv')
    period_promo_raw = pd.read_csv(p1_out_path,parse_dates=['Date'])

    seasonality_df = pd.read_csv('tmp/data/870_season.csv', parse_dates=['Date'])

    file_path = os.path.join(path, str(cate)+'/'+str(cate)+'_'+'ts.da')
    ts_df = pd.read_csv(file_path,header=None,sep='\t')
    model,feature=ow_train.train(area_rdc_map,period_promo_raw,promoCalendarDf,ts_df,scenario,holidays_df,seasonality_df,process_f01_flag=False,mode='dev',ForecastStartDate=pred_date)
    #feature.to_csv(os.path.join(result_path, str(cate)+'_train_feature_with_season_180201_0515.csv'),index=False)
    save_object(model, os.path.join(result_path, str(cate)+'/'+str(cate)+'_train_model.pkl'))


convert to Int columns:
Holiday
convert to Int columns:
Ind_1111_pre
convert to Int columns:
Ind_1111
convert to Int columns:
Ind_1111_post
convert to Int columns:
Ind_618_pre
convert to Int columns:
Ind_618
convert to Int columns:
Ind_618_post
convert to Int columns:
Ind_1212
convert to Float columns:
MaxSyntheticDiscountA
convert to Float columns:
MinSyntheticDiscountA
convert to Float columns:
AvgSyntheticDiscountA
convert to Float columns:
SyntheticGrossPrice
convert to Float columns:
promotionkey
convert to Float columns:
promotiontype
convert to Float columns:
promotionsubtype
convert to Float columns:
syntheticgrossprice_vb
convert to Float columns:
jdprice
convert to Float columns:
syntheticdiscounta_vb
convert to Float columns:
durationinhours
convert to Float columns:
daynumberinpromotion
convert to Float columns:
bundleflag
convert to Float columns:
directdiscountflag
convert to Float columns:
freegiftflag
convert to Float columns:
suitflag
convert to Float columns:
stockpri

ow_train.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  ts_df_pred.replace('null', np.nan, inplace=True)
ow_train.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  ts_df_pred.replace(-999, np.nan, inplace=True)


use cart feature?
False
| Add date features...
| - Add month of the year...
| - Add day of the week...
| Calculating national rolling features...
('| - Rolling value:', 'xxxHashColumn')
| - Rolling 14 mean...
| - Rolling 7 mean...
| - Rolling 5 mean...
| - Rolling 3 mean...
| - Rolling 2 mean...
| - Rolling 1 mean...
| - Rolling 14 median...
| - Rolling 7 median...
| - Rolling decay 28 mean...
| - Rolling decay 14 mean...
| - Rolling decay 7 mean...
| - Rolling decay 3 mean...
| - Added rolling features in 498.526178 seconds 

| Calculating  rolling features...
('| - Rolling value:', 'OrderNonOutlierVolume')
| - Rolling 360 mean...
| - Rolling 180 mean...
| - Rolling 90 mean...
| - Rolling 28 mean...
| - Rolling 14 mean...
| - Rolling 7 mean...
| - Rolling 5 mean...
| - Rolling 3 mean...
| - Rolling 2 mean...
| - Rolling 1 mean...
| - Rolling 14 median...
| - Rolling 7 median...
| - Rolling decay 360 mean...
| - Rolling decay 180 mean...
| - Rolling decay 90 mean...
| - Rolling decay 2

In [18]:
os.path.join(result_path, str(cate)+'/'+str(cate)+'_train_model.pkl')

'tmp/data/shishang/2589/2589_train_model.pkl'